In [ ]:
## To Download 100 images of each of three category
from bing_image_downloader import downloader
downloader.download('Green Parks', limit=100,  output_dir='dataset1',
dadult_filter_off=True, force_replace=False, timeout=60)

downloader.download('Deserts', limit=100,  output_dir='dataset1',
adult_filter_off=True, force_replace=False, timeout=60)

downloader.download('Sunsets', limit=100,  output_dir='dataset1',
adult_filter_off=True, force_replace=False, timeout=60)

In [ ]:
## Download Necessary Imports
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array, load_img
from skimage.io import imsave, imshow

In [ ]:
path = 'dataset\\'

#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

#Resize images
train = train_datagen.flow_from_directory(path, target_size=(256, 256), batch_size=512, class_mode=None)

In [ ]:
## Creating X, Y vectors
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

In [ ]:
def define_model(width, height, channels=1):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(width, height, channels)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

    #Decoder
    #Decoder
    #Note: For the last layer we use tanh instead of Relu. 
    #This is because we are colorizing the image in this layer using 2 filters, A and B.
    #A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
    #as it also has the range between -1 and 1. 
    #Other functions go from 0 to 1.
    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])

    return model

In [ ]:
colorize=define_model(256,256,1)
trained_model=colorize.fit(X,Y,validation_split=0.2, epochs=50, batch_size=16)
#colorize.save('colorize_autoencoder_50_epochs.model')

In [ ]:
import matplotlib.pyplot as plt
loss = trained_model.history['loss']
val_loss = trained_model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = trained_model.history['accuracy']
val_acc = trained_model.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
def predict_results(path):
    img1_grey=[]

    img1=img_to_array(load_img(path))
    img1 = resize(img1 ,(256,256))
    img1_grey.append(img1)

    img1_grey = np.array(img1_grey, dtype=float)
    img1_grey = rgb2lab(1.0/255*img1_grey)[:,:,:,0]
    img1_grey = img1_grey.reshape(img1_grey.shape+(1,))

    output1 = colorize.predict(img1_grey)
    output1 = output1*128

    result = np.zeros((256, 256, 3))
    result[:,:,0] = img1_grey[0][:,:,0]
    result[:,:,1:] = output1[0]

    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    ax = axes.ravel()
    ax[0].imshow(img1/255)
    ax[0].set_title('Grey Image')
    ax[1].imshow(lab2rgb(result))
    ax[1].set_title('Colored Image')

    fig.tight_layout()
    plt.show()
    return None

In [ ]:
predict_results('try images\img2.jpeg')